# Creación de un panel de inteligencia artificial responsable para evaluar los modelos

Al comparar y evaluar los modelos de Machine Learning, querrá revisar más que solo su métrica de rendimiento. Azure Machine Learning permite crear un panel de inteligencia artificial responsable para explorar cómo funciona el modelo en diferentes cohortes de los datos. 

## Preparación de los datos

Para crear el panel de inteligencia artificial responsable, necesitará un conjunto de datos de entrenamiento y prueba, almacenado como archivos Parquet y registrados como recursos de datos.

Los datos se almacenan actualmente como archivos CSV. Se convertirán en archivos Parquet. 


In [ ]:
## Antes de continuar:

Necesitará la versión más reciente del paquete **azureml-ai-ml** para ejecutar el código en este cuaderno. Ejecute la celda siguiente para comprobar que está instalada.

> **Nota**:
> Si el paquete **azure-ai-ml** no está instalado, ejecute `pip install azure-ai-ml` para instalarlo.

In [ ]:
## Conexión con su área de trabajo

Con los paquetes de SDK necesarios instalados, ya está listo para conectarse al área de trabajo.

Para conectarse a un espacio de trabajo, se necesitan parámetros de identificación: un id. de suscripción, un nombre de grupo de recursos y un nombre de espacio de trabajo. El nombre del grupo de recursos y el nombre del área de trabajo ya se rellenan automáticamente. Solo necesita el identificador de suscripción para completar el comando.

Para buscar los parámetros necesarios, haga clic en la suscripción y el nombre del área de trabajo en la parte superior derecha de Studio. Se abrirá un panel a la derecha.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copie el identificador de suscripción y reemplace **YOUR-SUBSCRIPTION-ID** por el valor que copió. </p>

## Creación de los recursos de datos

Para crear el panel de inteligencia artificial responsable, debe registrar los conjuntos de datos de entrenamiento y pruebas como recursos de datos **MLtable**. Los recursos de datos MLtable hacen referencia a los archivos Parquet que creó anteriormente.

In [ ]:
## Creación de la canalización para crear el panel de inteligencia artificial responsable

Para crear el panel, creará una canalización con componentes creados previamente registrados de forma predeterminada en el área de trabajo de Azure Machine Learning.

### Registro del modelo

Ya se ha entrenado un modelo de Machine Learning. El modelo predice si un paciente tiene diabetes. Todos los archivos de modelo se almacenan en la carpeta `model`. 

Registre el modelo apuntando a la carpeta `model` y su contenido.

In [ ]:
## Crear la canalización

Para crear el panel de inteligencia artificial responsable, creará una canalización mediante los componentes creados previamente. Puede elegir qué componentes usar y qué características desea incluir en el panel. Creará un panel que incluye el análisis de errores y la interpretación del modelo. 

Junto a las características de inteligencia artificial responsables, una canalización para crear un panel debe incluir un componente al principio para construir el panel y un componente al final para recopilar toda la información generada.

In [ ]:
Cuando haya recuperado todos los componentes que desea usar, puede compilar la canalización y conectar los componentes en el orden adecuado:

1. Construya el panel.
1. Agregue el análisis de errores.
1. Agregue explicaciones.
1. Recopile toda la información y visualícelas en el panel.

Ahora que se ha compilado la canalización, debe definir las dos entradas necesarias: el conjunto de datos de entrenamiento y prueba.

In [ ]:
Por último, se reunirá todo: asigne las entradas a la canalización y establezca la columna de destino (la etiqueta de predicción).

In [ ]:
## Ejecución de la canalización

Cuando haya compilado correctamente la canalización, puede enviarla. El código siguiente enviará la canalización y comprobará el estado de la canalización. También puede ver el estado de la canalización en Studio. 

Una vez completada la canalización, puede revisar el panel en Studio. 

In [ ]:
# Get handle to azureml registry for the RAI built in components
registry_name = "azureml"
ml_client_registry = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    registry_name=registry_name,
)
print(ml_client_registry)

### Register the model

A machine learning model has already been trained for you. The model predicts whether a patient has diabetes. All model files are stored in the `model` folder. 

Register the model by pointing to the `model` folder and its contents.

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

file_model = Model(
    path="model",
    type=AssetTypes.MLFLOW_MODEL,
    name="local-mlflow-diabetes",
    description="Model created from local file.",
)
model = ml_client.models.create_or_update(file_model)

## Build the pipeline

To create the responsible AI dashboard, you'll create a pipeline using the prebuilt components. You can choose which components to use, and which features you want to include in your dashboard. You'll create a dashboard that includes error analysis and model interpretability. 

Next to the responsible AI features, a pipeline to build a dashboard needs to include a component at the start to construct the dashboard, and a component at the end to gather all generated insights.

In [ ]:
model_name = model.name
expected_model_id = f"{model_name}:1"
azureml_model_id = f"azureml:{expected_model_id}"

In [ ]:
label = "latest"

rai_constructor_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_insight_constructor", label=label
)

# we get latest version and use the same version for all components
version = rai_constructor_component.version
print("The current version of RAI built-in components is: " + version)

rai_erroranalysis_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_erroranalysis", version=version
)

rai_explanation_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_explanation", version=version
)

rai_gather_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_insight_gather", version=version
)

When you've retrieved all components you want to use, you can build the pipeline and connect the components in the appropriate order:

1. Construct the dashboard.
1. Add error analysis.
1. Add explanations.
1. Gather all insights and visualize them in the dashboard.

In [ ]:
from azure.ai.ml import Input, dsl
from azure.ai.ml.constants import AssetTypes

compute_name = "aml-cluster"

@dsl.pipeline(
    compute=compute_name,
    description="RAI insights on diabetes data",
    experiment_name=f"RAI_insights_{model_name}",
)
def rai_decision_pipeline(
    target_column_name, train_data, test_data
):
    # Initiate the RAIInsights
    create_rai_job = rai_constructor_component(
        title="RAI dashboard diabetes",
        task_type="classification",
        model_info=expected_model_id,
        model_input=Input(type=AssetTypes.MLFLOW_MODEL, path=azureml_model_id),
        train_dataset=train_data,
        test_dataset=test_data,
        target_column_name=target_column_name,
    )
    create_rai_job.set_limits(timeout=30)

    # Add error analysis
    error_job = rai_erroranalysis_component(
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
    )
    error_job.set_limits(timeout=10)

    # Add explanations
    explanation_job = rai_explanation_component(
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
        comment="add explanation", 
    )
    explanation_job.set_limits(timeout=10)

    # Combine everything
    rai_gather_job = rai_gather_component(
        constructor=create_rai_job.outputs.rai_insights_dashboard,
        insight_3=error_job.outputs.error_analysis,
        insight_4=explanation_job.outputs.explanation,
    )
    rai_gather_job.set_limits(timeout=10)

    rai_gather_job.outputs.dashboard.mode = "upload"

    return {
        "dashboard": rai_gather_job.outputs.dashboard,
    }

Now the pipeline has been built, you need to define the two necessary inputs: the training and test dataset.

In [ ]:
from azure.ai.ml import Input
target_feature = "Diabetic"

diabetes_train_pq = Input(
    type="mltable",
    path=f"azureml:{input_train_data}:{data_version}",
    mode="download",
)
diabetes_test_pq = Input(
    type="mltable",
    path=f"azureml:{input_test_data}:{data_version}",
    mode="download",
)

Finally, we'll put everything together: assign the inputs to the pipeline and set the target column (the predicted label).

In [ ]:
import uuid
from azure.ai.ml import Output

# Pipeline to construct the RAI Insights
insights_pipeline_job = rai_decision_pipeline(
    target_column_name="Diabetic",
    train_data=diabetes_train_pq,
    test_data=diabetes_test_pq,
)

# Workaround to enable the download
rand_path = str(uuid.uuid4())
insights_pipeline_job.outputs.dashboard = Output(
    path=f"azureml://datastores/workspaceblobstore/paths/{rand_path}/dashboard/",
    mode="upload",
    type="uri_folder",
)


## Run the pipeline

When you've successfully built the pipeline, you can submit it. The following code will submit the pipeline and check the status of the pipeline. You can also view the pipeline's status in the studio. 

In [ ]:
from azure.ai.ml.entities import PipelineJob
from IPython.core.display import HTML
from IPython.display import display
import time

def submit_and_wait(ml_client, pipeline_job) -> PipelineJob:
    created_job = ml_client.jobs.create_or_update(pipeline_job)
    assert created_job is not None

    print("Pipeline job can be accessed in the following URL:")
    display(HTML('<a href="{0}">{0}</a>'.format(created_job.studio_url)))

    while created_job.status not in [
        "Completed",
        "Failed",
        "Canceled",
        "NotResponding",
    ]:
        time.sleep(30)
        created_job = ml_client.jobs.get(created_job.name)
        print("Latest status : {0}".format(created_job.status))
    assert created_job.status == "Completed"
    return created_job


# This is the actual submission
insights_job = submit_and_wait(ml_client, insights_pipeline_job)

When the pipeline is completed, you can review the dashboard in the studio. 